In [1]:
import cv2
import utils
import pickle
import ml_metrics

In [2]:
query_path_qsd2_w1 = 'datasets/qsd2_w1'
query_path_qsd1_w2 = 'datasets/qsd1_w2'
query_path_qsd2_w2 = 'datasets/qsd2_w2'

gt_path_qsd2_w1 = 'datasets/qsd2_w2/gt_corresps.pkl'

bbdd_path = 'datasets/BBDD'

# Task 1 - Calculate descriptors based on HSV

In [3]:
# Import and prepare query images
queries_2_w1 = utils.import_images(query_path_qsd2_w1)
queries_2_w1 = utils.convert_color_space(queries_2_w1, cv2.COLOR_BGR2HSV)
queries_2_w1 = utils.resize_images(queries_2_w1, (1024, 1024))
queries_2_w1 = sorted(queries_2_w1.items())
queries_2_w1 = dict(queries_2_w1)

In [5]:
bbdd = utils.import_images(bbdd_path)
bbdd = utils.convert_color_space(bbdd, cv2.COLOR_BGR2HSV)
bbdd = utils.resize_images(bbdd, (1024,1024))
bbdd = sorted(bbdd.items())
bbdd = dict(bbdd)

In [6]:
# Calculate CieLab histogram in number of blocks

blocks = [32, 16, 8, 4]
bins = 48

In [9]:
hist_blocks_queries_2 = utils.calculate_histogram_with_multiple_levels(queries_2_w1, blocks, bins)

In [10]:
hist_blocks_bbdd = utils.calculate_histogram_with_multiple_levels(bbdd, blocks, bins)

In [11]:
# Get the best k results for the intersection each query image with blocks and all BBDD for the first dataset

result_queries_with_blocks = []

for key_query in queries_2_w1.keys():
    hist_query = hist_blocks_queries_2[key_query]
    result_queries_with_blocks.append(utils.intersection_per_query(hist_query, hist_blocks_bbdd, 10))

In [12]:
gt_2_w1 = pickle.load(open('datasets/qsd2_w1/gt_corresps.pkl', 'rb', buffering=0), encoding='bytes')

In [13]:
# Result for DS 2 with blocks

eval = ml_metrics.mapk(gt_2_w1, result_queries_with_blocks, 1)
eval

0.4

In [14]:
# Result for DS 2 with blocks

eval = ml_metrics.mapk(gt_2_w1, result_queries_with_blocks, 5)
eval

0.43444444444444436

# Task 3 - Detect and remove text from images in QSD1-W2

In [15]:
# Import queries from dataset 1

queries_1_w2 = utils.import_images(query_path_qsd1_w2)
queries_1_w2 = sorted(queries_1_w2.items())
queries_1_w2 = dict(queries_1_w2)

In [16]:
# Import queries from dataset 2

queries_2_w2 = utils.import_images(query_path_qsd2_w2)
queries_2_w2 = sorted(queries_2_w2.items())
queries_2_w2 = dict(queries_2_w2)

In [17]:
# Import BBDD

bbdd = utils.import_images(bbdd_path)

In [18]:
# Import GT values for both datasets

gt_1_w2 = pickle.load(open('datasets/qsd1_w2/gt_corresps.pkl', 'rb', buffering=0), encoding='bytes')
gt_2_w2 = pickle.load(open('datasets/qsd2_w2/gt_corresps.pkl', 'rb', buffering=0), encoding='bytes')

In [19]:
with open('datasets/qsd1_w2/text_boxes.pkl', 'rb') as f:
    text_boxes = pickle.load(f)
boxes = []
for t in text_boxes:

    tlx = t[0][0][0]
    tly = t[0][0][1]
    brx = t[0][2][0]
    bry = t[0][2][1]
    new = [tlx,tly, brx,bry]
    boxes.append(new)


In [20]:
bboxes = utils.find_text(queries_1_w2, boxes)

# Task 4 - Evaluate the text bounding box

In [21]:
res = []
for b, t, in zip(bboxes.values(), text_boxes):
    tlx = t[0][0][0]
    tly = t[0][0][1]
    brx = t[0][2][0]
    bry = t[0][2][1]
    new = [tlx,tly, brx,bry]

    a = float(utils.iou(b,new))
    res.append(a)

print(sum(res)/len(res))

0.5705726269563006


# code for the mask (function find mask)

In [22]:
masks, images_masked = utils.find_mask_w2(queries_2_w2)
masks

{'00000': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 '00001': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 '00002': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 '00003': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 '00004': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, 

# Task 5

In [23]:
blocks = [32, 16, 8, 4]
bins = 48

queries_1_w2 = utils.import_images(query_path_qsd1_w2)
queries_1_w2 = utils.convert_color_space(queries_1_w2, cv2.COLOR_BGR2HSV)
queries_1_w2 = utils.resize_images(queries_1_w2, (1024, 1024))
queries_1_w2 = sorted(queries_1_w2.items())
queries_1_w2 = dict(queries_1_w2)

hist_blocks_queries_1_w2 = utils.calculate_histogram_with_multiple_levels(queries_1_w2, blocks, bins, with_bboxes=True, bboxes=bboxes)

In [24]:
result_queries_1_w2 = []

for key_query in queries_1_w2.keys():
    hist_query = hist_blocks_queries_1_w2[key_query]
    result_queries_1_w2.append(utils.intersection_per_query(hist_query, hist_blocks_bbdd, 10))

In [25]:
eval = ml_metrics.mapk(gt_1_w2, result_queries_1_w2, 1)
eval

0.5

In [26]:
eval = ml_metrics.mapk(gt_1_w2, result_queries_1_w2, 5)
eval

0.5444444444444445

# Task 6

In [27]:
queries_2_w2 = utils.import_images(query_path_qsd2_w2)
queries_2_w2 = sorted(queries_2_w2.items())
queries_2_w2 = dict(queries_2_w2)

In [28]:
blocks = [32, 16, 8, 4]
bins = 48

In [29]:
bbdd = utils.import_images(bbdd_path)
bbdd = sorted(bbdd.items())
bbdd = dict(bbdd)

In [30]:
hist_blocks_bbdd = utils.calculate_all_blocks(blocks, data = bbdd, bins=bins)

In [ ]:
final_histograms = {}
for key, value in masks.items():
    hist_per_image = []
    zeros = len(value==0) # get the amount of pixels that constitute non-painting from the mask

    # Apply mask to image to remove background
    result = cv2.bitwise_and(queries_2_w2[key], queries_2_w2[key], mask=value)
    result_gray = cv2.cvtColor(result, cv2.COLOR_BGRA2GRAY)
    contours, _ = cv2.findContours(result_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) >= 1:
        x,y,w,h = cv2.boundingRect(contours[0])
        ROI = result[y:y+h, x:x+w]

        # Find box
        bbox = utils.find_text({key: ROI}, boxes)

        ROI = cv2.cvtColor(ROI, cv2.COLOR_BGR2HSV)
        hist_per_image.append(utils.calculate_all_blocks(blocks, data={key: ROI}, bins=bins, with_bboxes=True, bboxes={key: bbox[key]})[key])

        if len(contours) == 2:
            x,y,w,h = cv2.boundingRect(contours[1])
            ROI = result[y:y+h, x:x+w]

            # Find box
            bbox = utils.find_text({key: ROI}, boxes)

            ROI = cv2.cvtColor(ROI, cv2.COLOR_BGR2HSV)
            hist_per_image.append(utils.calculate_all_blocks(blocks, data={key: ROI}, bins=bins, with_bboxes=True, bboxes={key: bbox[key]})[key])

    final_histograms[key] = hist_per_image

In [ ]:
import numpy as np

def intersection_per_query(hist_query, hist_bbdd: dict, k):
    result_query = {}

    if(np.shape(hist_query)[0] == 2):
        for key, value in hist_bbdd.items():
            result_query[key] = [utils.histogram_intersection(hist_query[0], value)]
            result_query[key].append([utils.histogram_intersection(hist_query[1], value)])

    best_k = dict(sorted(result_query.items(), key=lambda item: item[1], reverse=True)[:k])
    return [utils.extract_number_from_string(key) for key in best_k.keys()]

In [ ]:
result_queries_2_w2 = []

for key_query in queries_2_w2.keys():
    hist_query = final_histograms[key_query]

    result_queries_2_w2.append(intersection_per_query(hist_query, hist_blocks_bbdd, 10))